In [9]:
%pylab --no-import-all
from os import path
import pandas as pd

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
try:
    file = path.join("..", "data", "interim", "london.csv")
except OSError:
    print("This repository does not host the data. "
          "Put the csv in ../data/interim/")
    raise
df = pd.read_csv(file, na_values=0)

Rename Italian columns to English.

In [11]:
df.rename(columns={'DONNA': 'ID',
                   'P_SPEZZ': 'GROUP_ID',
                   'P_CICLO': 'CYCLE_ID',
                   'ANNO_NAS': 'BIRTH_YR',
                   'DATA': 'BEGIN_DATE',
                   'T_SPEZZ': 'N_GROUPS',
                   'T_CICLI': 'N_CYCLES',
                   'QUALIFI': 'DESC',
                   'TIPOTEMP': 'TEMP_SCALE',
                   'L_CICLO': 'L_CYCLE',
                   'L_PREOV': 'L_PREOVULATION',
                   'L_PERIOD': 'L_PERIOD',
                   'FIGLI': 'CHILDREN'
                  },
         inplace=True)

We delete the cycles (rows) where the `L_PREOV` is missing (NA), because this data is unusable - we don't have the variable we seek to predict. **Ex:** If row 100 is deleted, the index would be $..., 98, 99, 101, 102, ...$

After these rows are dropped, there are holes created in the index of the data frame. We would like the index numbers to be consecutive, so we use the `reset_index` function.

In [12]:
df.dropna(subset=['L_PREOVULATION'], inplace=True)

Delete those where DESC != 1 because those are flawed entries.

In [13]:
df = df[df.DESC == 1]
df.drop('DESC', 1, inplace=True)


Decode temperatures to obtain *real* BBT values. Then, convert all celsius values to fahrenheit.

In [30]:
FAHR = 1
CELS = 2
for i in range (1,100):
    df.ix[df.TEMP_SCALE == FAHR, 'TEMP' + str(i)] =  90 + df.ix[df.TEMP_SCALE == FAHR, 'TEMP' + str(i)]/10
    df.ix[df.TEMP_SCALE == CELS, 'TEMP' + str(i)] =  36 + df.ix[df.TEMP_SCALE == CELS , 'TEMP' + str(i)]/10
    #Convert celsius temps to fahrenheit
    #df.ix[df.TEMP_SCALE == CELS, 'TEMP' + str(i)] =  32 + ((9/5) *  df.ix[df.TEMP_SCALE == CELS , 'TEMP' + str(i)])

#Display medians one at a time    
#df[df.TEMP_SCALE==FAHR].median()
df[df.TEMP_SCALE==CELS].median()

ID                7861.0
GROUP_ID             1.0
CYCLE_ID            10.0
BIRTH_YR            46.0
N_GROUPS             1.0
N_CYCLES            21.0
TEMP_SCALE           2.0
L_CYCLE             27.0
L_PREOVULATION      15.0
L_PERIOD             5.0
TEMP1               40.0
TEMP2               40.0
TEMP3               40.0
TEMP4               40.0
TEMP5               40.0
TEMP6               40.0
TEMP7               40.0
TEMP8               40.0
TEMP9               40.0
TEMP10              40.0
TEMP11              40.0
TEMP12              40.0
TEMP13              40.0
TEMP14              40.0
TEMP15              40.0
TEMP16              40.0
TEMP17              40.0
TEMP18              40.0
TEMP19              40.0
TEMP20              40.0
                   ...  
TEMP72              40.0
TEMP73              40.0
TEMP74              40.0
TEMP75              40.0
TEMP76              40.0
TEMP77              40.0
TEMP78              40.0
TEMP79              40.0
TEMP80               NaN


Create an age measurement for each cycle, computed from the birth year and measurement date.

In [29]:
df['AGE'] = (df.BEGIN_DATE.apply(lambda s: int(s.split('-')[-1]) if isinstance(s, str) else s) - df.BIRTH_YR)

In [16]:
import errno
import os

destination = ["..", "data", "processed"]
try:
    os.makedirs(path.join(*destination))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
df.to_csv(path.join(*destination, "df.csv"))